In [98]:
import json
import random
import uuid

!pip install -q psycopg2
import psycopg2

In [99]:
connection.close()
cursor.close()

In [100]:
# Connect to the postgres server linked to the container this notebook is running in
connection = psycopg2.connect("host=postgres dbname=wisdom user=postgres")

In [101]:
# Create the patients, reports and snps tables
with connection.cursor() as cursor:
    cursor.execute("""
        drop table if exists patients;
        create table patients (id varchar(64) not null primary key, age int not null, sign varchar(32) not null);
        drop table if exists reports;
        create table reports (id varchar(64) not null primary key, report jsonb not null);
        drop table if exists snps;
        create table snps (id varchar(64) not null primary key, snps jsonb not null);
        """)
    connection.commit()

In [102]:
# Load the sample report and snps as a template
with open("samples/sample_color_gene_report.json") as f:
    report_template = json.loads(f.read())["data"]
with open("samples/sample_color_snp_report.json") as f:
    snps_template = json.loads(f.read())["data"]

In [103]:
# Create a bunch of synthetic patient, report, and snps and add them to the database
for i in range(250):
    with connection.cursor() as cursor:
        patient_id = str(uuid.uuid4())
        sign = ['Capricorn', 'Sagittarius', 'Scorpio', 'Libra',
                'Virgo', 'Leo', 'Cancer', 'Gemini', 
                'Taurus', 'Aries', 'Pisces', 'Aquarius', 'Capricorn'][random.randint(0,12)]
        age = random.randint(18,90)
        cursor.execute("""insert into patients values ('{}', {}, '{}')""".format(patient_id, age, sign))

        # Copy the template report and pick a random subset of 8 variants
        report = report_template.copy()
        report["patient_id"] = patient_id
        report["variants"] = random.sample(report_template["variants"], 8)

        cursor.execute("""insert into reports values ('{}', '{}')""".format(str(uuid.uuid4()), json.dumps(report)))

        # snps all the same as we expect all for each record, now to mix it up?
        snps = snps_template.copy()
        snps["patient_id"] = patient_id
        cursor.execute("""insert into snps values ('{}', '{}')""".format(str(uuid.uuid4()), json.dumps(snps)))

        connection.commit()